In [1]:
import numpy as np
import pandas as pd
import sys
import os
import duckdb

In [2]:
sys.path.append('..')
from helper import *

setwd()
Objects = Objects()

In [3]:
Objects

[{'id': 'eurostat',
  'format': 'xlsx',
  'landing_temp_name': 'prc_hicp_midx__custom_.*_spreadsheet.xlsx',
  'landing_temp_tblID': 'prc_hicp_midx__custom_(.*)_spreadsheet.xlsx'},
 {'id': 'ine',
  'format': 'csv',
  'landing_temp_name': '50918.csv',
  'landing_temp_tblID': 'tbl001'}]

In [11]:
# connection to duckdb
con = duckdb.connect(database="data/formatted/db_formatted.db", read_only=False)

In [10]:
con.close()

In [ ]:
# con.execute()

In [ ]:
con.execute("")

## number format for csv's

change commas by dots